In [1]:
# https://keras.io/
!pip install -q keras
import keras

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 10
num_filter = 20

Using TensorFlow backend.


In [2]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 89s 1us/step


In [0]:
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=8)

In [0]:
def create_block(x):
  layer = Conv2D(32, (3,3), strides=(1,1), padding='same', use_bias=False)(x)
  layer = BatchNormalization()(layer)
  layer = LeakyReLU(alpha=0.1)(layer)

  layer = Conv2D(64, (3,3), strides=(1,1), padding='same', use_bias=False)(layer)
  layer = BatchNormalization()(layer)
  layer = LeakyReLU(alpha=0.1)(layer)
  
  layer = Conv2D(128, (3,3), strides=(1,1), padding='same', use_bias=False)(layer)
  layer = BatchNormalization()(layer)
  layer = LeakyReLU(alpha=0.1)(layer)
  
  layer = Conv2D(256, (3,3), strides=(1,1), padding='same', use_bias=False)(layer)
  layer = BatchNormalization()(layer)
  layer = LeakyReLU(alpha=0.1)(layer)
  
  layer = MaxPooling2D(pool_size=(2, 2))(layer)
  return layer  

In [5]:
input = Input(shape=(img_height, img_width, channel,))

block1 = create_block(input)
skip_connection = block1
block2 = create_block(block1)
block3 = create_block(block2)
block4 = create_block(block3)


skip_connection = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_21', use_bias=False)(skip_connection)
skip_connection = BatchNormalization(name='norm_21')(skip_connection)
skip_connection = LeakyReLU(alpha=0.1)(skip_connection)
skip_connection = Lambda(space_to_depth_x2)(skip_connection)

x = concatenate([skip_connection, block4])

flatten = Flatten()(x)

output = Dense(num_classes, activation='softmax')(flatten)



Instructions for updating:
Colocations handled automatically by placer.


In [6]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 32, 32, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [8]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 112s 2ms/step - loss: 1.4700 - acc: 0.5247 - val_loss: 1.2006 - val_acc: 0.6036
Epoch 2/50
50000/50000 [==============================] - 104s 2ms/step - loss: 0.9120 - acc: 0.6863 - val_loss: 1.0477 - val_acc: 0.6610
Epoch 3/50
50000/50000 [==============================] - 104s 2ms/step - loss: 0.7471 - acc: 0.7424 - val_loss: 1.1411 - val_acc: 0.6426
Epoch 4/50
50000/50000 [==============================] - 104s 2ms/step - loss: 0.6246 - acc: 0.7850 - val_loss: 1.0888 - val_acc: 0.6534
Epoch 5/50
50000/50000 [==============================] - 104s 2ms/step - loss: 0.5179 - acc: 0.8191 - val_loss: 1.0061 - val_acc: 0.6865
Epoch 6/50
50000/50000 [==============================] - 104s 2ms/step - loss: 0.4208 - acc: 0.8535 - val_loss: 0.9801 - val_acc: 0.6942
Epoch 7/50
50000/50000 [==============================] - 104s 2ms/step - 

In [9]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Save the trained weights in to .h5 format
model.save_weights("ASSIGNMENT_3C.h5")
print("Saved the model to disk")

10000/10000 [==============================] - 8s 783us/step
Test loss: 2.2345538202285766
Test accuracy: 0.7113
Saved the model to disk


In [10]:
model.save_weights("ASSIGNMENT_3C.h5")
print("Saved the model to disk")
from google.colab import files

files.download('ASSIGNMENT_3C.h5')

Saved the model to disk
